In [37]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)


-test-

In [58]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './image_30/train',
        target_size = (300,300),
        batch_size=10,
        color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './image_30/test',
        classes = {},
        target_size=(300,300),
        batch_size=5,
        color_mode ='rgb',

        shuffle =True,
        seed = 42,
        class_mode='categorical')

Found 150 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [ ]:

'''

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './image_30/train',
        classes = {}
        target_size=(300,300),
        batch_size=100,
        color_moder ='rgb'
        shuffle =True
        seed = 42
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
        classes = {}
        target_size=(300,300),
        batch_size=100,
        color_moder ='rgb'
        shuffle =True
        seed = 42
        class_mode='categorical')
'''

In [52]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=5,
        epochs=10,
        validation_data=test_generator,
        validation_steps=10)

model.save('model-5')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Epoch 1/10
5/5 [==============================] - 2s 377ms/step - loss: 1.6261 - accuracy: 0.3000 - val_loss: 1.2151 - val_accuracy: 0.3200
Epoch 2/10
5/5 [==============================] - 2s 304ms/step - loss: 1.1227 - accuracy: 0.2800 - val_loss: 1.0480 - val_accuracy: 0.4200
Epoch 3/10
5/5 [==============================] - 2s 303ms/step - loss: 1.0492 - accuracy: 0.5000 - val_loss: 1.0650 - val_accuracy: 0.3400
Epoch 4/10
5/5 [==============================] - 2s 345ms/step - loss: 1.0202 - accuracy: 0.4800 - val_loss: 1.0312 - val_accuracy: 0.4800
Epoch 5/10
5/5 [==============================] - 2s 312ms/step - loss: 0.9811 - accuracy: 0.5000 - val_loss: 1.0068 - val_accuracy: 0.5600
Epoch 6/10
5/5 [==============================] - 2s 312ms/step - loss: 0.9022 - accuracy: 0.5400 - val_loss: 0.9622 - val_accuracy: 0.5000
Epoch 7/10
5/5 [==============================] - 2s 354ms/step - loss: 0.7074 - accuracy: 0.7200 - val_loss: 1.0613 - val_accuracy: 0.4600
Epoch 8/10
5/5 [====

In [53]:
model.summary()
model.save('model-5')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 147, 147, 16)      4624      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 85264)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2728480   
_________________________________________________________________
dense_13 (Dense)             (None, 32)               

<class 'tensorflow.python.keras.preprocessing.image.DirectoryIterator'>


In [54]:
def predict_image_sample(model,X_test, y_test, test_id= -1 ):
    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(10000)
    else:
        test_sample_id = test_id
    
    test_image =X_test[test_sample_id]
#     plt.imshow(test_images[123])

    
    test_image = test_image.reshape(1,300,300,3)
    
    y_actual = y_test[test_sample_id].tolist().index(1)
    print('y_actual number = ', y_actual)
    
    y_pred = model.predict(test_image)
    print("y_pred = ", y_pred)
    
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    print('y_pred number = ', y_pred_num)
    return y_actual , y_pred_num


In [55]:
def test_load_data():
    train_images = []       
    train_labels = []
    shape = (300,300)  
    train_path = './image_test'

    for classes in ['f','i','e']:
        for filename in os.listdir('./image_test'):
            if classes == filename[0]:
                img = cv2.imread(os.path.join(train_path,filename))
                train_labels.append(filename[2:0:-1])

                # Resize all images to a specific shape
                #img = cv2.resize(img,shape)

                train_images.append(img)
            else:
                pass

    # Converting labels into One Hot encoded sparse matrix

    # 음식, 실내 ,실외 순으로 들어가기 위해 index 처리
    # train_labels = sorted(train_labels)
    # print(train_labels)

    train_labels = pd.get_dummies((train_labels)).values
    # print(train_labels)
    # Converting train_images to array
    train_images = np.array(train_images)

    # Splitting Training data into train and validation dataset

    train_images ,train_labels= shuffle(train_images ,train_labels, random_state = 42) 

    return train_images, train_labels

In [57]:
if __name__ == '__main__' :
    n = 0
    
    X_test , y_test = test_load_data()
    model = load_model('model-5')
    for i in range(185):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', i,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)
    
    

y_actual number =  0
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  2
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  2
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0.000 1.000 0.000]]
y_pred numb

y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  2
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  2
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  2
y_pred =  [[0.000 1.000 0.000]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0.000 0.000 1.000]]
y_pred number =  2
y_actual numb